In [ ]:
from torchvision import transforms
from PIL import Image
import torch
from vgg import Vgg16  
import utils

style_img = Image.open("style.png").convert("RGB")
style_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.mul(255))
])
style_tensor = style_transform(style_img).unsqueeze(0)
vgg = Vgg16(requires_grad=False)
style_features = vgg(utils.normalize_batch(style_tensor))
style_grams = [utils.gram_matrix(f) for f in style_features]

frame_img = Image.open("1.png").convert("RGB")
frame_tensor = style_transform(frame_img).unsqueeze(0)

frame_features = vgg(utils.normalize_batch(frame_tensor))

loss_fn = torch.nn.MSELoss()
style_loss = 0.
for f_f, g_s in zip(frame_features, style_grams):
    g_f = utils.gram_matrix(f_f)
    style_loss += loss_fn(g_f, g_s)

print("Style consistency loss:", style_loss.item())


Style consistency loss: 0.0010487308027222753
